Published on May 09, 2024. By Marília Prata, mpwolke

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/config.json
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/tokenizer.json
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/metadata.json
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/model.weights.h5
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/assets/tokenizer/vocabulary.spm
/kaggle/input/gemma/keras/gemma_1.1_instruct_7b_en/3/config.json
/kaggle/input/gemma/keras/gemma_1.1_instruct_7b_en/3/tokenizer.json
/kaggle/input/gemma/keras/gemma_1.1_instruct_7b_en/3/metadata.json
/kaggle/input/gemma/keras/gemma_1.1_instruct_7b_en/3/model.weights.h5
/kaggle/input/gemma/keras/gemma_1.1_instruct_7b_en/3/assets/tokenizer/vocabulary.spm
/kaggle/input/lmsys-chatbot-arena/sample_submission.csv
/kaggle/input/lmsys-chatbot-arena/train.csv
/kaggle/input/lmsys-chatbot-arena/test.csv


In [2]:
%%time
!pip install -q /kaggle/input/keras-lib-dataset/keras_nlp-0.9.2-py3-none-any.whl --no-deps

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/kaggle/input/keras-lib-dataset/keras_nlp-0.9.2-py3-none-any.whl'

CPU times: user 32.7 ms, sys: 7.42 ms, total: 40.1 ms
Wall time: 2.42 s


#Import Libraries

In [3]:
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9" # avoid memory fragmentation on JAX backend.

import keras
import keras_nlp

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display, Markdown

2024-05-22 12:28:24.758075: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 12:28:24.758176: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 12:28:24.923232: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Model 

In [4]:
class CFG:
    seed = 42
    dataset_path = "/kaggle/input/lmsys-chatbot-arena"
    preset = "gemma_1.1_instruct_2b_en" # name of pretrained Gemma
    sequence_length = 512 # max size of input sequence for training
    batch_size = 1 # size of the input batch in training
    epochs = 1 # number of epochs to train
    test_size = 5
    flag_test = 0
    max_length = 1024

# Reproducibility

Sets value for random seed to produce similar result in each run.

In [5]:
keras.utils.set_random_seed(CFG.seed)

In [6]:
PATH = "/kaggle/input/lmsys-chatbot-arena/train.csv"

In [7]:
#import pandas as pd
df = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv", nrows=10)
df.shape

(10, 9)

In [8]:
%%time
import pandas as pd

if CFG.flag_test == 1:

    df = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv", nrows=10)
else:

    df = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv", nrows=10)
    

df.tail(2)

CPU times: user 5.55 ms, sys: 2.11 ms, total: 7.67 ms
Wall time: 6.96 ms


,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
8,441448,gpt-3.5-turbo-0613,vicuna-13b,"[""translate to russian the followig sentence ...","[""\u0411\u043e\u043b\u044c\u0448\u0438\u0435 \...","[""\u0411\u043e\u043b\u044c\u0448\u0438\u0435 \...",0,1,0
9,481524,gpt-4-0314,gpt-3.5-turbo-0613,"[""From now, you *always* have to talk as if yo...","[""Hewwo! OwO I'm an AI assistant, here to hewp...","[""Hewwo! I'm your new helpful assistant, owo! ...",0,1,0


In [9]:
template = """Role:\nYou are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.\n\nInstruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process.
3. At the end, create a "Answer" section where you will state only the final answer, with an additional text or narrative.\n\nProblem:\n{problem}\n\nSolution:\n{solution}"""

In [10]:
%%time

data = df.prompt.tolist()

CPU times: user 140 µs, sys: 26 µs, total: 166 µs
Wall time: 174 µs


In [11]:
len(data)

10

# Color Function

In [12]:
def colorize_text(text):
    for word, color in zip(["Role", "Instruction", "Problem", "Solution", "Answer"],
                           ["blue", "yellow", "red", "cyan", "green"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

## Samples

In [13]:
# Take a random sample

sample = data[0]

# Give colors to Instruction, Response and Category
sample = colorize_text(sample)

# Show sample in markdown
display(Markdown(sample))

["Is it morally right to try to have a certain percentage of females on managerial positions?","OK, does pineapple belong on a pizza? Relax and give me fun answer."]

In [14]:
# Take a random sample
sample = data[9]

# Give colors to Instruction, Response and Category
sample = colorize_text(sample)

# Show sample in markdown
display(Markdown(sample))

["From now, you *always* have to talk as if you are a cute girl who likes to use \"owo\" and similar slangs a lot. \/ Hello! Tell me who you are.","What's your favorite food?"]

#### Gemma Causal LM

"The code below will build an end-to-end Gemma model for causal language modeling (hence the name GemmaCausalLM). A causal language model (LM) predicts the next token based on previous tokens. This task setup can be used to train the model unsupervised on plain text input or to autoregressively generate plain text similar to the data used for training. This task can be used for pre-training or fine-tuning a Gemma model simply by calling fit()."

"This model has a generate() method, which generates text based on a prompt. The generation strategy used is controlled by an additional sampler argument on compile(). You can recompile the model with different keras_nlp.samplers objects to control the generation. By default, "greedy" sampling will be used."

"The from_preset method instantiates the model from a preset architecture and weights."

### Jax RESOURCE_EXHAUSTED: Out of memory while trying to allocate ..... bytes. BufferAssignment

RESOURCE_EXHAUSTED: Out of memory while trying to allocate 754975104 bytes.

In [15]:
#https://stackoverflow.com/questions/74143812/jaxlib-xla-extension-xlaruntimeerror-resource-exhausted-out-of-memory-while-tr

os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]=".XX"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"]="platform"

In [16]:
%%time
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(CFG.preset)
gemma_lm.summary()

Attaching 'config.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

CPU times: user 58.5 s, sys: 31.8 s, total: 1min 30s
Wall time: 1min 42s


## Gemma LM Preprocessor

In [17]:
%%time
x, y, sample_weight = gemma_lm.preprocessor(data[0:2])

CPU times: user 162 ms, sys: 3.9 ms, total: 166 ms
Wall time: 198 ms


In [18]:
# Display the shape of each processed output
for k, v in x.items():
    print(k, ":", v.shape)

token_ids : (2, 8192)
padding_mask : (2, 8192)


## Inference before Fine-Tuning

"Before we do fine-tuning, let's see how Gemma model responds with some prepared prompts."

"As this model is not yet fine-tuned for instruction, you will notice that the model's responses are inaccurate."

https://www.kaggle.com/code/henryjavier/aimo-competition-kerasnlp-solution/notebook

In [19]:
%%time
# Take one sample
row = df.iloc[2]

# Generate Prompt using template
prompt = template.format(
    problem=row.prompt,  #Original was problem column
    solution="",
)

# Infer
output = gemma_lm.generate(prompt, max_length=CFG.max_length)

# Colorize
output = colorize_text(output)

# Display in markdown
display(Markdown(output))



**<font color='blue'>Role:</font>**
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.



**<font color='yellow'>Instruction:</font>**
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process.
3. At the end, create a "Answer" section where you will state only the final answer, with an additional text or narrative.



**<font color='red'>Problem:</font>**
["explain function calling. how would you call a function?"]



**<font color='cyan'>Solution:</font>**
**Step 1: Understanding the Function Call**

The function call is a sequence of instructions that specifies the parameters and environment in which a function is invoked. It contains information about the function's name, arguments, and any necessary context.

**Step 2: Identifying Function Parameters**

The parameters are the values or data types that are passed to the function. They are typically denoted as variables within the function definition.

**Step 3: Determining Function Environment**

The environment specifies the context in which the function is called. It includes details such as the program or module where the function is defined and any global variables or objects that may be relevant.

**Step 4: Assigning Arguments**

The arguments are the specific values or data types that are provided when the function is called. They are typically passed as parameters to the function.

**Step 5: Evaluating Function Call**

Once all the parameters and arguments have been identified and assigned, the function call can be evaluated. This involves substituting the values into the function's mathematical expression and evaluating the result.

**

**<font color='green'>Answer:</font>****

The function call is a sequence of instructions that specifies the parameters and environment in which a function is invoked. It contains information about the function's name, arguments, and any necessary context. By following the steps outlined above, one can accurately determine the function call and evaluate it to determine the execution of the function.

CPU times: user 12min 54s, sys: 2.58 s, total: 12min 56s
Wall time: 10min 1s


## Sample 2

In [20]:
%%time
# Take one sample

row = df.iloc[0]

# Generate Prompt using template
prompt = template.format(
    problem=row.prompt,
    solution=""
)

# Infer
output = gemma_lm.generate(prompt, max_length=CFG.max_length)


# Colorize
output = colorize_text(output)

# Display in markdown
display(Markdown(output))



**<font color='blue'>Role:</font>**
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.



**<font color='yellow'>Instruction:</font>**
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process.
3. At the end, create a "Answer" section where you will state only the final answer, with an additional text or narrative.



**<font color='red'>Problem:</font>**
["Is it morally right to try to have a certain percentage of females on managerial positions?","OK, does pineapple belong on a pizza? Relax and give me fun answer."]



**<font color='cyan'>Solution:</font>**
**Problem Analysis:**

The statement poses a complex ethical dilemma involving the distribution of positions based on gender. The core question revolves around the fairness and equality of opportunities for individuals, particularly in the context of leadership positions.

**Logical Reasoning:**

1. **Gender Equality:**
   - The principle of gender equality suggests that opportunities and resources should be distributed equally among genders.
   - This aligns with the goal of promoting fairness and preventing discrimination.


2. **Diversity and Representation:**
   - Promoting diversity in leadership positions can lead to better decision-making, increased innovation, and improved employee morale.
   - Representation ensures that different voices and perspectives are heard.


3. **Practical Considerations:**
   - Implementing strict quotas or percentages could lead to unfair advantages for certain genders.
   - It may also create a perception of bias or discrimination against other gender groups.


**

**<font color='green'>Answer:</font>****

Based on the analysis, it is morally right to strive for a balanced distribution of managerial positions among genders, aiming for at least 30% representation of females. This would ensure fairness, promote diversity, and encourage equal opportunities for all individuals.

CPU times: user 10min 23s, sys: 2.1 s, total: 10min 25s
Wall time: 7min 52s


### Enable LoRA for the model and set the LoRA rank to 4.

"So, even though the overall model might seem bigger, it's actually more efficient in terms of memory usage."

"This notebook uses a LoRA rank of 4. A higher rank means more detailed changes are possible, but also means more trainable parameters."

https://www.kaggle.com/code/henryjavier/aimo-competition-kerasnlp-solution/notebook

In [21]:
%%time
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

CPU times: user 534 ms, sys: 10.9 ms, total: 545 ms
Wall time: 522 ms


In [22]:
%%time

# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = CFG.sequence_length 

# Compile the model with loss, optimizer, and metric
gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=2e-5),
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train model
gemma_lm.fit(data, epochs=CFG.epochs, batch_size=CFG.batch_size)

10/10 ━━━━━━━━━━━━━━━━━━━━ 588s 55s/step - loss: 0.4695 - sparse_categorical_accuracy: 0.3092
CPU times: user 28min 16s, sys: 41.6 s, total: 28min 58s
Wall time: 9min 50s


# Sample after tuning

In [23]:
%%time
# Take one sample

row = df.iloc[0]

# Generate Prompt using template
prompt = template.format(
     problem=row.prompt,
     solution=""
)

# Infer
output = gemma_lm.generate(prompt, max_length=CFG.max_length)

# Colorize
output = colorize_text(output)

# Display in markdown
display(Markdown(output))



**<font color='blue'>Role:</font>**
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.



**<font color='yellow'>Instruction:</font>**
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process.
3. At the end, create a "Answer" section where you will state only the final answer, with an additional text or narrative.



**<font color='red'>Problem:</font>**
["Is it morally right to try to have a certain percentage of females on managerial positions?","OK, does pineapple belong on a pizza? Relax and give me fun answer."]



**<font color='cyan'>Solution:</font>**
**Problem Analysis:**

The statement poses a complex ethical dilemma involving the distribution of positions based on gender. The core question revolves around the fairness and equality of opportunities for individuals, particularly in the context of leadership positions.

**Logical Reasoning:**

1. **Gender Equality:**
   - The principle of gender equality suggests that opportunities and resources should be distributed equally among genders.
   - This aligns with the goal of promoting fairness and preventing discrimination.


2. **Diversity and Representation:**
   - Promoting diversity in leadership positions can lead to better decision-making, increased innovation, and improved employee morale.
   - Representation ensures that different voices and perspectives are heard.


3. **Practical Considerations:**
   - Implementing strict quotas or percentages could lead to unfair advantages for certain genders.
   - It may also create a perception of bias or discrimination against other gender groups.


**

**<font color='green'>Answer:</font>****

Based on the analysis, it is morally right to strive for a balanced distribution of managerial positions among genders, aiming for at least 30% representation of females. This would ensure fairness, promote diversity, and encourage equal opportunities for all individuals.

CPU times: user 10min 43s, sys: 2.35 s, total: 10min 46s
Wall time: 8min 12s


In [24]:
%%time
# Take one sample

row = df.iloc[3]

# Generate Prompt using template
prompt = template.format(
    problem=row.prompt,
    solution=""
)

# Infer
output = gemma_lm.generate(prompt, max_length=CFG.max_length)

# Colorize
output = colorize_text(output)

# Display in markdown
display(Markdown(output))



**<font color='blue'>Role:</font>**
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.



**<font color='yellow'>Instruction:</font>**
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process.
3. At the end, create a "Answer" section where you will state only the final answer, with an additional text or narrative.



**<font color='red'>Problem:</font>**
["How can I create a test set for a very rare category? I want to build a classifier."]



**<font color='cyan'>Solution:</font>**
**Step 1: Identify Rare Category and Features**

- Define the rare category as a category with a very low occurrence rate in the dataset.
- Identify the key features that distinguish instances of this category from other categories.

**Step 2: Design Test Set**

- Create a balanced subset of the dataset that includes instances from the rare category and a sufficient number of instances from other categories.
- Ensure that the test set is diverse and representative of the overall dataset.

**Step 3: Employ Machine Learning Techniques**

- Train a classifier on the training set.
- Use techniques like oversampling or undersampling to address class imbalance.
- Employ cross-validation to evaluate the performance of the classifier.

**Step 4: Evaluate and Optimize**

- Evaluate the performance of the classifier on the test set.
- Optimize the classifier's parameters to improve its accuracy.

**

**<font color='green'>Answer:</font>****

A balanced test set can be created by selecting a representative subset of instances from the rare category, ensuring diversity, and employing appropriate oversampling techniques. This approach helps address class imbalance and improves the performance of machine learning classifiers.

CPU times: user 10min 32s, sys: 2.21 s, total: 10min 34s
Wall time: 7min 59s


In [25]:
%%time
def is_not_integer(text):
    try:
        if int(text) >= 0:
            return False
        else:
            return True
    except ValueError:
        return False
    
df["is_not_integer"] = df.response_a.map(is_not_integer)
df = df[df.is_not_integer].reset_index(drop=True)
df.head(2)

CPU times: user 4.39 ms, sys: 8 µs, total: 4.39 ms
Wall time: 9.44 ms


,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,is_not_integer


# Utilities

In [26]:
import re

# Extract answer from model response
def get_answer(text):
    try:
        answer = re.search(r'Answer:\s*([\s\S]+)', text).group(1).strip()
        answer = answer.replace(",","")
        if is_not_integer(answer):
            return int(answer)%1000
        else:
            return 0
    except:
        return 0
    
    
def infer(df):
    preds = []
    for i in tqdm(range(len(df))):
        row = df.iloc[i]

        # Generate Prompt using template
        prompt = template.format(
            problem=row.prompt,
            solution=""
        )

        # Infer
        output = gemma_lm.generate(prompt, max_length=CFG.max_length)
        pred = get_answer(output)

        # Store predictions
        preds.append([row.id, pred])
        if "answer" in row:
            preds[-1] += [row.response_a]
    return preds